In [1]:
!pip install stable-baselines3[extra]

     -------------------------------------- 181.7/181.7 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 953.9/953.9 kB 2.4 MB/s eta 0:00:00
     -------------------------------------- 192.3/192.3 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 10.8/10.8 MB 3.2 MB/s eta 0:00:00
     -------------------------------------- 434.7/434.7 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     -------------------------------------- 952.4/952.4 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import gym
from stable_baselines3 import PPO # RL algorithm
from stable_baselines3.common.vec_env import DummyVecEnv # train agents on multiple envs
from stable_baselines3.common.evaluation import evaluate_policy

import matplotlib.pyplot as plt
from IPython import display

In [2]:
# openAI gym allows you to build simulated envs easily
env_name = "CartPole-v0"
env = gym.make(env_name, render_mode="rgb_array")

c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render() # view the graphical representation of the env

        # plt.imshow(env.render())
        # display.display(plt.gcf())    
        # display.clear_output(wait=True)

        action = env.action_space.sample()
        n_state, reward, done, truncated, info = env.step(action) # apply the action to the env
        score += reward
    
    print(f"Episode:{episode} Score: {score}")
env.close()

Episode:1 Score: 39.0
Episode:2 Score: 20.0
Episode:3 Score: 31.0
Episode:4 Score: 45.0
Episode:5 Score: 25.0


### Train the RL model

In [7]:
log_path = os.path.join('Training', 'Logs')

In [8]:
env = gym.make(env_name, render_mode="rgb_array")
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device


In [9]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1


c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


-----------------------------
| time/              |      |
|    fps             | 249  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 248         |
|    iterations           | 2           |
|    time_elapsed         | 16          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009032854 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000966   |
|    learning_rate        | 0.0003      |
|    loss                 | 6.5         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0177     |
|    value_loss           | 56.4        |
-----------------------------------------
----------------------------------

### Save and reload the model

In [ ]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')
model.save(PPO_Path)
model = PPO.load(PPO_Path, env=env)

In [12]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [13]:
env.close()

### Test the model

In [17]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render() # view the graphical representation of the env

        action, _ = model.predict(obs)
        obs, reward, done, truncated = env.step(action) # apply the action to the env
        score += reward
    
    print(f"Episode:{episode} Score: {score}")
env.close()

Episode:1 Score: [200.]
Episode:2 Score: [200.]
Episode:3 Score: [200.]
Episode:4 Score: [200.]
Episode:5 Score: [200.]


### View logs in tensorboard

In [18]:
training_log_path = os.path.join(log_path, 'PPO_1')
training_log_path

'Training\\Logs\\PPO_1'

In [ ]:
!tensorboard --logdir={training_log_path}

In [ ]:
# Core metrics to look at
## Average reward
## Average episode length

# Training stratergoes
## Train for longer
## Hyper parameter tuning
## Try different algorithms

### Adding a callback to the traning stage

In [20]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [19]:
save_path = os.path.join('Training', 'Saved Models')

In [21]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, callback_on_new_best=stop_callback, eval_freq=10000, best_model_save_path= save_path, verbose=1)

In [22]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [23]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 392  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 303         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008092849 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00431    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.016      |
|    value_loss           | 55.9        |
-----------------------------------------
---

c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=195.80 +/- 8.40
Episode length: 195.80 +/- 8.40
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 196         |
|    mean_reward          | 196         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009076485 |
|    clip_fraction        | 0.084       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.605      |
|    explained_variance   | 0.323       |
|    learning_rate        | 0.0003      |
|    loss                 | 21.2        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0185     |
|    value_loss           | 55.4        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 261   |
|    iterations      | 5     |
|    time

### Changing Policies (Optional)

In [24]:
NN_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [25]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch':NN_arch})

Using cuda device


c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [26]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 256  |
|    iterations      | 1    |
|    time_elapsed    | 7    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 217         |
|    iterations           | 2           |
|    time_elapsed         | 18          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014647996 |
|    clip_fraction        | 0.198       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00516     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.48        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0238     |
|    value_loss           | 20.3        |
-----------------------------------------
---

c:\Users\ranuja\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=194.60 +/- 10.80
Episode length: 194.60 +/- 10.80
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 195         |
|    mean_reward          | 195         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009562263 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.577      |
|    explained_variance   | 0.458       |
|    learning_rate        | 0.0003      |
|    loss                 | 16.9        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 44.3        |
-----------------------------------------
------------------------------
| time/              |       |
|    fps             | 204   |
|    iterations      | 5     |
|    time_elapsed    | 50    

### Use an alternate algorithm

In [27]:
from stable_baselines3 import DQN

In [28]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [29]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.97     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2006     |
|    time_elapsed     | 0        |
|    total_timesteps  | 63       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.932    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3110     |
|    time_elapsed     | 0        |
|    total_timesteps  | 143      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.889    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 3268     |
|    time_elapsed     | 0        |
|    total_timesteps  | 234      |
----------------------------------
------------------------